In [27]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
ruta='/content/drive/MyDrive/Ingeniería y Ciencia de Datos/Segundo Semestre/Ingeniería de Características/Colab Notebooks/'
data=pd.read_csv(ruta+'temps.txt',sep='\s+',header=None) # Fuente: http://astro.iam.udg.mx/Estacion/NOAAPRMO.TXT
                                                         #         http://astro.iam.udg.mx/Estacion/NOAAMO.TXT
data.columns=['Temperatura máxima','Temperatura mínima','HDD','CDD','Rain']
d=['Sábado','Domingo','Lunes','Martes','Miércoles','Jueves','Viernes']
dias=[d[i%7] for i in range(len(data))]
catego=['Soleado','Mayormente soleado','Parcialmente nublado','Mayormente nublado','Nublado','Lluvioso']
prono=[catego[-1] if data['Rain'][i]>0 else catego[int(round(1.2*(data['HDD'][i]-data['CDD'][i])+2))] for i in range(len(data))]
df=pd.DataFrame(columns=['Días','Pronóstico','Temperatura máxima','Temperatura mínima'],index=range(len(data)))
df.iloc[:,2:4]=data.iloc[:,0:2]
df['Días']=dias
df['Pronóstico']=prono
df.head()

txt1="Pedro Perez Pereira, pobre pintor portuguez, pinta preciosos paisajes para poder partir para Paris."
txt2="Preciosos gatos y perros con mancha quiero. Partir de izquierda a derecha para poder llegar a algun lugar."
txt3="En un lugar de la Mancha, cuyo nombre no quiero acordarme..."

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Codificación por conteo

## Codificación de días

In [6]:
conteo_Dias=df['Días'].value_counts()
conteo_Dias

Sábado       7
Domingo      7
Lunes        7
Martes       7
Miércoles    7
Jueves       7
Viernes      6
Name: Días, dtype: int64

In [7]:
df['Conteo_Días']=df['Días'].map(conteo_Dias)
df.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,Conteo_Días
0,Sábado,Parcialmente nublado,28.2,11.2,7
1,Domingo,Parcialmente nublado,29.1,10.7,7
2,Lunes,Mayormente nublado,26.1,10.4,7
3,Martes,Mayormente soleado,29.4,11.1,7
4,Miércoles,Mayormente soleado,29.8,11.1,7


## Codificación de pronóstico

In [8]:
conteo_Pronostico=df['Pronóstico'].value_counts()
conteo_Pronostico

Parcialmente nublado    18
Mayormente soleado      14
Mayormente nublado       6
Soleado                  6
Lluvioso                 2
Nublado                  2
Name: Pronóstico, dtype: int64

In [9]:
df['Conteo_Pronóstico']=df['Pronóstico'].map(conteo_Pronostico)
df.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,Conteo_Días,Conteo_Pronóstico
0,Sábado,Parcialmente nublado,28.2,11.2,7,18
1,Domingo,Parcialmente nublado,29.1,10.7,7,18
2,Lunes,Mayormente nublado,26.1,10.4,7,6
3,Martes,Mayormente soleado,29.4,11.1,7,14
4,Miércoles,Mayormente soleado,29.8,11.1,7,14


In [10]:
df['fred_Pronóstico']=df['Pronóstico'].map(conteo_Pronostico/len(df))
df.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,Conteo_Días,Conteo_Pronóstico,fred_Pronóstico
0,Sábado,Parcialmente nublado,28.2,11.2,7,18,0.375000
1,Domingo,Parcialmente nublado,29.1,10.7,7,18,0.375000
2,Lunes,Mayormente nublado,26.1,10.4,7,6,0.125000
3,Martes,Mayormente soleado,29.4,11.1,7,14,0.291667
4,Miércoles,Mayormente soleado,29.8,11.1,7,14,0.291667


# Bag of words

In [11]:
txt1,txt2,txt3


('Pedro Perez Pereira, pobre pintor portuguez, pinta preciosos paisajes para poder partir para Paris.',
 'Preciosos gatos y perros con mancha quiero. Partir de izquierda a derecha para poder llegar a algun lugar.',
 'En un lugar de la Mancha, cuyo nombre no quiero acordarme...')

In [12]:
txt1.split()

['Pedro',
 'Perez',
 'Pereira,',
 'pobre',
 'pintor',
 'portuguez,',
 'pinta',
 'preciosos',
 'paisajes',
 'para',
 'poder',
 'partir',
 'para',
 'Paris.']

In [13]:
re.split('\W+',txt1) # '\W'      ->  caracteres especiales (espacios, tabular, puntos, coma)
                    # '\w'      ->  caracteres alfanuméricos
                    # '\d'      ->  caracteres numéricos
                    # '\s'      ->  espacios en blanco
                    # [a-z]     ->  letras minúsculas
                    # ^[0-9]    ->  caracteres no numéricos
                    # a{2,4}    ->  el caracter 'a' de 2 a 4 veces
                    # . (punto) ->  cualquier caracter excepto cambio de línea

['Pedro',
 'Perez',
 'Pereira',
 'pobre',
 'pintor',
 'portuguez',
 'pinta',
 'preciosos',
 'paisajes',
 'para',
 'poder',
 'partir',
 'para',
 'Paris',
 '']

In [14]:
palabras=re.split('\W+',(txt1+txt2+txt3).lower())
palabras

['pedro',
 'perez',
 'pereira',
 'pobre',
 'pintor',
 'portuguez',
 'pinta',
 'preciosos',
 'paisajes',
 'para',
 'poder',
 'partir',
 'para',
 'paris',
 'preciosos',
 'gatos',
 'y',
 'perros',
 'con',
 'mancha',
 'quiero',
 'partir',
 'de',
 'izquierda',
 'a',
 'derecha',
 'para',
 'poder',
 'llegar',
 'a',
 'algun',
 'lugar',
 'en',
 'un',
 'lugar',
 'de',
 'la',
 'mancha',
 'cuyo',
 'nombre',
 'no',
 'quiero',
 'acordarme',
 '']

In [15]:
re.findall('para',txt1, flags=re.I) #re.I -> Ignore case

['para', 'para']

In [16]:
set_palabras=set(palabras)
set_palabras.remove('')
set_palabras

{'a',
 'acordarme',
 'algun',
 'con',
 'cuyo',
 'de',
 'derecha',
 'en',
 'gatos',
 'izquierda',
 'la',
 'llegar',
 'lugar',
 'mancha',
 'no',
 'nombre',
 'paisajes',
 'para',
 'paris',
 'partir',
 'pedro',
 'pereira',
 'perez',
 'perros',
 'pinta',
 'pintor',
 'pobre',
 'poder',
 'portuguez',
 'preciosos',
 'quiero',
 'un',
 'y'}

In [17]:
d_palabra={}
for palabra in set_palabras:
  n_txt1=txt1.lower().count(palabra)
  n_txt2=txt2.lower().count(palabra)
  n_txt3=txt3.lower().count(palabra)
  d_palabra[palabra]=[n_txt1,n_txt2,n_txt3]
pd.DataFrame(d_palabra,index=['txt1','txt2','txt3'])

,acordarme,y,mancha,a,nombre,paris,portuguez,lugar,gatos,cuyo,...,poder,perez,de,paisajes,no,preciosos,para,derecha,la,partir
txt1,0,0,0,10,0,1,1,0,0,0,...,1,1,1,1,0,1,2,0,0,1
txt2,0,1,1,13,0,0,0,1,1,0,...,1,0,3,0,0,1,1,1,0,1
txt3,1,1,1,6,1,0,0,1,0,1,...,0,0,1,0,2,0,0,0,1,0


In [18]:
_['mancha']

txt1    0
txt2    1
txt3    1
Name: mancha, dtype: int64

In [19]:
conteo=CountVectorizer()
m_conteo=conteo.fit_transform([txt1,txt2,txt3])
m_conteo

<3x31 sparse matrix of type '<class 'numpy.int64'>'
	with 39 stored elements in Compressed Sparse Row format>

In [20]:
pd.DataFrame(m_conteo.toarray(),columns=conteo.get_feature_names_out(),index=['txt1','txt2','txt3'])

,acordarme,algun,con,cuyo,de,derecha,en,gatos,izquierda,la,...,perez,perros,pinta,pintor,pobre,poder,portuguez,preciosos,quiero,un
txt1,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,1,1,1,0,0
txt2,0,1,1,0,1,1,0,1,1,0,...,0,1,0,0,0,1,0,1,1,0
txt3,1,0,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,1


In [21]:
set_palabras-set(conteo.get_feature_names_out())

{'a', 'y'}

In [22]:
conteo=CountVectorizer(ngram_range=(2,2))
m_conteo=conteo.fit_transform([txt1,txt2,txt3])
pd.DataFrame(m_conteo.toarray(),columns=conteo.get_feature_names_out(),index=['txt1','txt2','txt3'])

,algun lugar,con mancha,cuyo nombre,de izquierda,de la,derecha para,en un,gatos perros,izquierda derecha,la mancha,...,pintor portuguez,pobre pintor,poder llegar,poder partir,portuguez pinta,preciosos gatos,preciosos paisajes,quiero acordarme,quiero partir,un lugar
txt1,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,0,1,0,0,0
txt2,1,1,0,1,0,1,0,1,1,0,...,0,0,1,0,0,1,0,0,1,0
txt3,0,0,1,0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,1


In [25]:
set_palabras-set(conteo.get_feature_names_out())

{'a',
 'acordarme',
 'algun',
 'con',
 'cuyo',
 'de',
 'derecha',
 'en',
 'gatos',
 'izquierda',
 'la',
 'llegar',
 'lugar',
 'mancha',
 'no',
 'nombre',
 'paisajes',
 'para',
 'paris',
 'partir',
 'pedro',
 'pereira',
 'perez',
 'perros',
 'pinta',
 'pintor',
 'pobre',
 'poder',
 'portuguez',
 'preciosos',
 'quiero',
 'un',
 'y'}

In [26]:
conteo=CountVectorizer(ngram_range=(2,2))
m_conteo=conteo.fit_transform([txt1,txt2,txt3])
pd.DataFrame(m_conteo.toarray(),columns=conteo.get_feature_names_out(),index=['txt1','txt2','txt3'])

,algun lugar,con mancha,cuyo nombre,de izquierda,de la,derecha para,en un,gatos perros,izquierda derecha,la mancha,...,pintor portuguez,pobre pintor,poder llegar,poder partir,portuguez pinta,preciosos gatos,preciosos paisajes,quiero acordarme,quiero partir,un lugar
txt1,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,0,1,0,0,0
txt2,1,1,0,1,0,1,0,1,1,0,...,0,0,1,0,0,1,0,0,1,0
txt3,0,0,1,0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,1


In [28]:
tfidf=TfidfVectorizer()
m_tfidf=tfidf.fit_transform([txt1,txt2,txt3])
pd.DataFrame(m_tfidf.toarray(),columns=tfidf.get_feature_names_out(),index=['txt1','txt2','txt3'])

,acordarme,algun,con,cuyo,de,derecha,en,gatos,izquierda,la,...,perez,perros,pinta,pintor,pobre,poder,portuguez,preciosos,quiero,un
txt1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.276831,0.000000,0.276831,0.276831,0.276831,0.210537,0.276831,0.210537,0.000000,0.000000
txt2,0.000000,0.293267,0.293267,0.000000,0.223037,0.293267,0.000000,0.293267,0.293267,0.000000,...,0.000000,0.293267,0.000000,0.000000,0.000000,0.223037,0.000000,0.223037,0.223037,0.000000
txt3,0.327673,0.000000,0.000000,0.327673,0.249204,0.000000,0.327673,0.000000,0.000000,0.327673,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.249204,0.327673


In [29]:
_['para']

txt1    0.421074
txt2    0.223037
txt3    0.000000
Name: para, dtype: float64

In [30]:
tfidf=TfidfVectorizer(stop_words=['y','en','la','de','a']) # stop_word  ->  Palabras que no aportan significado a la oración
m_tfidf=tfidf.fit_transform([txt1,txt2,txt3])
pd.DataFrame(m_tfidf.toarray(),columns=tfidf.get_feature_names_out(),index=['txt1','txt2','txt3'])

,acordarme,algun,con,cuyo,derecha,gatos,izquierda,llegar,lugar,mancha,...,perez,perros,pinta,pintor,pobre,poder,portuguez,preciosos,quiero,un
txt1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.276831,0.000000,0.276831,0.276831,0.276831,0.210537,0.276831,0.210537,0.000000,0.000000
txt2,0.000000,0.300845,0.300845,0.000000,0.300845,0.300845,0.300845,0.300845,0.228800,0.228800,...,0.000000,0.300845,0.000000,0.000000,0.000000,0.228800,0.000000,0.228800,0.228800,0.000000
txt3,0.385323,0.000000,0.000000,0.385323,0.000000,0.000000,0.000000,0.000000,0.293048,0.293048,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.293048,0.385323
